In [18]:
import pandas as pd
import openpyxl
import numpy as np
import xlwings as xw


get the AVL from excel

In [19]:
AVL_name="BOM_AVL_362216.xlsx"
Manu_List_Name = "Manufacturer_362216.xlsx"

Manufacturer_Excel_Name = "tc_import_odm_supplier.xlsm"
OrderCode_Excel_Name = "tc_import_order_code.xlsm"
SupplierPart_Excel_Name = "tc_import_supplier_part.xlsm"

In [20]:
df_AVL = pd.read_excel(AVL_name)
df_Manu = pd.read_excel(Manu_List_Name)

Check Manufacturer ID match

In [21]:
#extract manf ID in AVL
df_Manf_ID = df_Manu[["ODM_Manufacturer_Distributor_ID"]]
df_Manf_ID = df_Manf_ID.drop_duplicates(subset="ODM_Manufacturer_Distributor_ID")
df_Manf_ID = df_Manf_ID.reset_index(drop=True)
print(f"There are {len(df_Manf_ID)} unique Manufacturer_IDs in AVL")

#extract manf ID in TC file and check matches 
df_manu_id_TC_raw = pd.read_excel(Manufacturer_Excel_Name, sheet_name="Sheet1")
df_manu_id_TC = df_manu_id_TC_raw.loc[:, ['Home', 'Object']]
df_manu_id_TC["Manf_ID"] = df_manu_id_TC["Object"].str.split('-', n=2).str[1]
df_manu_id_TC

There are 21 unique Manufacturer_IDs in AVL


Home                                             Object  \
0      1  362216_ODM-20002866-ARMA OFSET BASIM VE MATBAA...   
1      2         362216_ODM-20002478-TESA BANT SAN.TİC. A.Ş   
2      3    362216_ODM-60001383-ELECTRO TERMİNAL GMBH&CO KG   
3      4  362216_ODM-60000419-ÖZDİSAN ELEKTRONİK PAZARLA...   
4      5  362216_ODM-60000359-PRİNTRONİCS PCB ELEK.SAN.V...   
5      6  362216_ODM-20000826-TEMİZ SERİGRAFİ PLAS.MONTA...   
6      7             362216_ODM-YAGEOCORP-YAGEO CORPORATION   
7      8                   362216_ODM-20001273-CEREN ZENGER   
8      9      362216_ODM-60000786-NEUMÜLLER ELEKTRONİK GMBH   
9     10      362216_ODM-60000296-ARROW ELEKTRONİK TİC.A.Ş.   
10    11                  362216_ODM-KYOCERAAVX-KYOCERA AVX   
11    12  362216_ODM-60000134-ODAK PCB ELEKTRONİK ANONİM...   
12    13       362216_ODM-20001854-ICAPE HK COMPANY LIMITED   
13    14   362216_ODM-SEOULSEMIC-SEOUL SEMICONDUCTOR CO LTD   
14    15  362216_ODM-20001260-STC ENDÜSTRİYEL TİC. VE SA...   
15    16       362216_Vendor-ALPPLAS ENDUSTRIYEL YATIRIMLAR   
16    17  362216_ODM-60001391-CHANGZHOU AOHONG ELECTRONİ...   
17    18  362216_ODM-60000879-EBV ELEKTRONİK TİCARET LTD...   
18    19                   362216_ODM-LITTELFUSE-LITTELFUSE   
19    20                    362216_ODM-OSRAMGMBH-OSRAM GMBH   
20    21  362216_ODM-10000028-ALPPLAS ENDÜSTRİYEL YATIRI...   
21    22           362216_ODM-60001165-UPT OPTİK WODAK GMBH   

                           Manf_ID  
0                         20002866  
1                         20002478  
2                         60001383  
3                         60000419  
4                         60000359  
5                         20000826  
6                        YAGEOCORP  
7                         20001273  
8                         60000786  
9                         60000296  
10                      KYOCERAAVX  
11                        60000134  
12                        20001854  
13                      SEOULSEMIC  
14                        20001260  
15  ALPPLAS ENDUSTRIYEL YATIRIMLAR  
16                        60001391  
17                        60000879  
18                      LITTELFUSE  
19                       OSRAMGMBH  
20                        10000028  
21                        60001165

In [22]:
#extract manf ID in AVL
df_Manf_ID = df_Manu[["ODM_Manufacturer_Distributor_ID"]]
df_Manf_ID = df_Manf_ID.drop_duplicates(subset="ODM_Manufacturer_Distributor_ID")
df_Manf_ID = df_Manf_ID.reset_index(drop=True)
print(f"There are {len(df_Manf_ID)} unique Manufacturer_IDs in AVL")

#extract manf ID in TC file and check matches 
df_manu_id_TC_raw = pd.read_excel(Manufacturer_Excel_Name, sheet_name="Sheet1")
df_manu_id_TC = df_manu_id_TC_raw.loc[:, ['Home', 'Object']]
df_manu_id_TC["Manf_ID"] = df_manu_id_TC["Object"].str.split('-', n=2).str[1]
df_manu_id_TC[""] = ""
df_Manf_ID["ODM_Manufacturer_Distributor_ID"] = df_Manf_ID["ODM_Manufacturer_Distributor_ID"].astype(str)
df_manu_id_TC["Manf_ID_in_AVL"] = df_Manf_ID["ODM_Manufacturer_Distributor_ID"].str.replace(" ", "")


#save df to output excel
app = xw.App(visible=False)
output = xw.Book()
output_sheet = output.sheets.add("Sheet2")
output_sheet.clear_contents()
output_sheet.range('A1').value = df_manu_id_TC.columns.tolist()
output_sheet.range('A2').value = df_manu_id_TC.values

#write a formula to excel
num_rows = output_sheet.range('E2').expand().shape[0]
formula_range = output_sheet.range('F2:F{}'.format(num_rows+1))
formula_range.formula = '=IFNA(IF(MATCH(E2,$C$2:$C$3000,0),TRUE),"FALSE")'

#save the excel

output.save('manufacturerdistributor_id_tc_import_output.xlsm')
output.close()
app.quit()
print("Excel Saved, please check the matches there")

There are 21 unique Manufacturer_IDs in AVL
Excel Saved, please check the matches there


Check Order Code Match

In [23]:
#extract Order_Codes in AVL
df_OrderCode = df_AVL[["Order_Code","Manufacturer_ID"]]
df_OrderCode["Order_Code"] = df_OrderCode["Order_Code"].astype(str).str.replace(" ", "")
df_OrderCode["Manufacturer_ID"] = df_OrderCode["Manufacturer_ID"].astype(str)
df_OrderCode["Manufacturer_ID"] = df_OrderCode["Manufacturer_ID"].str.replace(" ", "")
df_OrderCode["Combined"] = df_OrderCode[["Order_Code","Manufacturer_ID"]].astype(str).agg("-".join,axis=1)

df_OrderCode = df_OrderCode.drop_duplicates(subset="Combined")
df_OrderCode = df_OrderCode.reset_index(drop=True)
print(f"There are {len(df_OrderCode)} unique Order_Codes in AVL")

#extract order_code in TC file and check matches 
#df_ordercode_TC = pd.read_excel("order_code_manufacturer_part_tc_import.xlsm", sheet_name="Sheet1")
df_ordercode_TC_raw = pd.read_excel(OrderCode_Excel_Name, sheet_name="Sheet1")
df_ordercode_TC = df_ordercode_TC_raw.loc[:, ['Home', 'Object', 'Order Code', 'Vendor ID']]
df_ordercode_TC["Combined_TC"] = df_ordercode_TC["Order Code"].str.replace(" ", "") + "-" + df_ordercode_TC["Vendor ID"].str.split("ODM-").str[1]
df_ordercode_TC[""] = ""
df_ordercode_TC["OrderCode_in_AVL"] = df_OrderCode["Combined"]

#save df to output excel
app = xw.App(visible=False)
output2 = xw.Book()
output_sheet2 = output2.sheets.add("Sheet2")
output_sheet2.clear_contents()
output_sheet2.range('A1').value = df_ordercode_TC.columns.tolist()
output_sheet2.range('A2').value = df_ordercode_TC.values

#write a formula to excel
num_rows = output_sheet2.range('G2').expand().shape[0]
formula_range = output_sheet2.range('H2:H{}'.format(num_rows+1))
formula_range.formula = '=IF(COUNTIF($E$2:$E$3000, G2)>0, TRUE, FALSE)'

#save the excel
output2.save('order_code_tc_import_output.xlsm')
output2.close()
app.quit()
print("Excel Saved, please check the matches there")

There are 73 unique Order_Codes in AVL


C:\Users\Scheele\AppData\Local\Temp\ipykernel_16768\3308396751.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_OrderCode["Order_Code"] = df_OrderCode["Order_Code"].astype(str).str.replace(" ", "")
C:\Users\Scheele\AppData\Local\Temp\ipykernel_16768\3308396751.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_OrderCode["Manufacturer_ID"] = df_OrderCode["Manufacturer_ID"].astype(str)
C:\Users\Scheele\AppData\Local\Temp\ipykernel_16768\3308396751.py:5: SettingWithCopyWarning: 
A value is trying to 

Excel Saved, please check the matches there


Check component number supplier part

In [24]:
#extract ODM_Component_Number in AVL
df_ODM_Component_Number = df_AVL[["ODM_Component_Number"]]
df_ODM_Component_Number = df_ODM_Component_Number.drop_duplicates(subset="ODM_Component_Number")
df_ODM_Component_Number = df_ODM_Component_Number.reset_index(drop=True)
print(f"There are {len(df_ODM_Component_Number)} unique ODM_Component_Number in AVL")

#extract ODM_Component_Number in TC file and check matches 
df_component_number_TC_raw = pd.read_excel(SupplierPart_Excel_Name, sheet_name="Sheet1")
df_component_number_TC = df_component_number_TC_raw.loc[:, ['Home', 'Object']]
df_component_number_TC["ComponentNum_in_TC"] = df_component_number_TC['Object'].str.extract(r'\d+\s(.+?)/')
df_component_number_TC[""] = ""
df_component_number_TC["Component_Number_in_AVL"] = df_ODM_Component_Number["ODM_Component_Number"]
df_component_number_TC

#save df to output excel
app = xw.App(visible=False)
output3 = xw.Book()
output_sheet3 = output3.sheets.add("Sheet2")
output_sheet3.clear_contents()
output_sheet3.range('A1').value = df_component_number_TC.columns.tolist()
output_sheet3.range('A2').value = df_component_number_TC.values

#write a formula to excel
num_rows = output_sheet3.range('E2').expand().shape[0]
formula_range = output_sheet3.range('F2:F{}'.format(num_rows+1))
formula_range.formula = '=IFNA(IF(MATCH(E2,$C$2:$C$3000,0),TRUE),"FALSE")'

#save the excel
output3.save('supplier_part_tc_import_output.xlsm')
output3.close()
app.quit()
print("Excel Saved, please check the matches there")


There are 72 unique ODM_Component_Number in AVL
Excel Saved, please check the matches there
